In [78]:
import requests
import bs4
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

In [119]:
url_inicial = 'http://rmcg.geociencias.unam.mx/index.php/rmcg/issue/archive?issuesPage=4#issues'
url_root = 'http://rmcg.geociencias.unam.mx/index.php/rmcg/issue/archive?issuesPage=4#issues'
r=requests.get(url_inicial)

In [120]:
soup = BeautifulSoup(r.text, 'html.parser')

In [121]:
box = soup.find('ul', class_='issues_archive')
volumen=box.findAll('div',class_='obj_issue_summary')
vol = [x.find('a').get('href')for x in volumen]
vol

['http://rmcg.geociencias.unam.mx/index.php/rmcg/issue/view/82',
 'http://rmcg.geociencias.unam.mx/index.php/rmcg/issue/view/83',
 'http://rmcg.geociencias.unam.mx/index.php/rmcg/issue/view/86',
 'http://rmcg.geociencias.unam.mx/index.php/rmcg/issue/view/87',
 'http://rmcg.geociencias.unam.mx/index.php/rmcg/issue/view/84',
 'http://rmcg.geociencias.unam.mx/index.php/rmcg/issue/view/85',
 'http://rmcg.geociencias.unam.mx/index.php/rmcg/issue/view/90',
 'http://rmcg.geociencias.unam.mx/index.php/rmcg/issue/view/91',
 'http://rmcg.geociencias.unam.mx/index.php/rmcg/issue/view/88',
 'http://rmcg.geociencias.unam.mx/index.php/rmcg/issue/view/89',
 'http://rmcg.geociencias.unam.mx/index.php/rmcg/issue/view/92',
 'http://rmcg.geociencias.unam.mx/index.php/rmcg/issue/view/93',
 'http://rmcg.geociencias.unam.mx/index.php/rmcg/issue/view/94',
 'http://rmcg.geociencias.unam.mx/index.php/rmcg/issue/view/95',
 'http://rmcg.geociencias.unam.mx/index.php/rmcg/issue/view/97',
 'http://rmcg.geociencias

In [122]:
vol2 =[]
for i in vol: 
    url_inicial1=i 
    r1 = requests.get(url_inicial1)
    soup1 = BeautifulSoup(r1.text, 'html.parser')
    box1 = soup1.find('ul', class_='articles') 
    volumen1=box1.findAll('div', class_='obj_article_summary')
    vol1 = [x.find('a').get('href')for x in volumen1]
    vol2+=vol1

In [123]:
vol2

['http://rmcg.geociencias.unam.mx/index.php/rmcg/article/view/1178',
 'http://rmcg.geociencias.unam.mx/index.php/rmcg/article/view/1179',
 'http://rmcg.geociencias.unam.mx/index.php/rmcg/article/view/1180',
 'http://rmcg.geociencias.unam.mx/index.php/rmcg/article/view/1181',
 'http://rmcg.geociencias.unam.mx/index.php/rmcg/article/view/1182',
 'http://rmcg.geociencias.unam.mx/index.php/rmcg/article/view/1183',
 'http://rmcg.geociencias.unam.mx/index.php/rmcg/article/view/1184',
 'http://rmcg.geociencias.unam.mx/index.php/rmcg/article/view/1185',
 'http://rmcg.geociencias.unam.mx/index.php/rmcg/article/view/1186',
 'http://rmcg.geociencias.unam.mx/index.php/rmcg/article/view/1187',
 'http://rmcg.geociencias.unam.mx/index.php/rmcg/article/view/1189',
 'http://rmcg.geociencias.unam.mx/index.php/rmcg/article/view/1197',
 'http://rmcg.geociencias.unam.mx/index.php/rmcg/article/view/1198',
 'http://rmcg.geociencias.unam.mx/index.php/rmcg/article/view/1199',
 'http://rmcg.geociencias.unam.mx/

In [124]:
def get_url_items(sopa,url):
    box1 = soup1.find('ul', class_='articles') 
    volumen1=box1.findAll('div', class_='obj_article_summary')
    vol1 = [x.find('a').get('href')for x in volumen1]
    return vol2

In [125]:
links_items=[]
i=0
while i<1:
    i+=1
    print(f'Estoy en la pagina {url_inicial}')
    r_pag=requests.get(url_inicial)
    s_p=BeautifulSoup(r_pag.text,'html.parser')
    links=get_url_items(s_p, url_inicial)
    links_items.append(links) 
    next_a=s_p.select('li.next > a')
    if not next_a or not next_a[0].get('href'):
        break

Estoy en la pagina http://rmcg.geociencias.unam.mx/index.php/rmcg/issue/archive?issuesPage=4#issues


In [126]:
list_scraper=[]
for i in links_items:
    for j in i:
        list_scraper.append(j)
len(list_scraper)

261

In [127]:
uno=list_scraper[0]
r_item=requests.get(uno)
s_item=BeautifulSoup(r_item.text, 'html.parser')

In [128]:
#funcion para iniciar el escraper de cada libro
def scraper_book(url):
    content_book={}
    r=requests.get(url)
    tire='Revista Mexicana de Ciencias Geológicas'
    a='I. Físico Matemáticas y Ciencias de la Tierra'
    tem='Geociencias'
    s_item=BeautifulSoup(r.text,'html.parser')
    #titulo de revista
    try:
        titulo=tire
        content_book['Titulo Revista']=tire
    except AttributeError:
        content_book['Titulo Revista']=None
    #area
    try:
        area=a
        content_book['Área']=a
    except AttributeError:
        content_book['Área']=None
    #tematica
    try:
        tema=tem
        content_book['Temática']=tem
    except AttributeError:
        content_book['Temática']=None
    #titulo articulo
    try:
        titu=s_item.find('h1', class_='page_title').get_text(strip=True)
        content_book['Titulo Articulo']=titu
    except AttributeError:
        content_book['Titulo Articulo']=None
    #resumen
    try:
        resu=s_item.find('div', class_='item cover_image').get_text(strip=True)
        content_book['Resumen']=resu
    except AttributeError:
        content_book['Resumen']=None
    #abstract
    try:
        abst=s_item.find('p').get_text(strip=True)
        content_book['Abstract']=abst
    except AttributeError:
        content_book['Abstract']=None
    #Link incial
    try:
        linkart=s_item.find('div', class_='sub_item').find('a').get('href')
        content_book['Link Articulo']=urljoin(url_root, linkart)
    except AttributeError:
        content_book['Link Articulo']=None
    #link articulo
    try:
        link=s_item.find('a', class_='obj_galley_link pdf').get('href')
        content_book['Link PDF']=urljoin(url_root, link)
    except AttributeError:
        content_book['Link PDF']=None
    return content_book

In [130]:
list_scraper=list_scraper[0:261]
datos_book=[]
for idx, i in enumerate(list_scraper):
    print(f'estas escrapeando la pag {idx}')
    datos_book.append(scraper_book(i))

estas escrapeando la pag 0
estas escrapeando la pag 1
estas escrapeando la pag 2
estas escrapeando la pag 3
estas escrapeando la pag 4
estas escrapeando la pag 5
estas escrapeando la pag 6
estas escrapeando la pag 7
estas escrapeando la pag 8
estas escrapeando la pag 9
estas escrapeando la pag 10
estas escrapeando la pag 11
estas escrapeando la pag 12
estas escrapeando la pag 13
estas escrapeando la pag 14
estas escrapeando la pag 15
estas escrapeando la pag 16
estas escrapeando la pag 17
estas escrapeando la pag 18
estas escrapeando la pag 19
estas escrapeando la pag 20
estas escrapeando la pag 21
estas escrapeando la pag 22
estas escrapeando la pag 23
estas escrapeando la pag 24
estas escrapeando la pag 25
estas escrapeando la pag 26
estas escrapeando la pag 27
estas escrapeando la pag 28
estas escrapeando la pag 29
estas escrapeando la pag 30
estas escrapeando la pag 31
estas escrapeando la pag 32
estas escrapeando la pag 33
estas escrapeando la pag 34
estas escrapeando la pag 35
es

In [131]:
df_catalogo=pd.DataFrame(datos_book)
df_catalogo

,Titulo Revista,Área,Temática,Titulo Articulo,Resumen,Abstract,Link Articulo,Link PDF
0,Revista Mexicana de Ciencias Geológicas,I. Físico Matemáticas y Ciencias de la Tierra,Geociencias,"Geology of the Sierra of Charcas, State of San...",None,The mapped area is 130 km north of the city of...,http://rmcg.geociencias.unam.mx/index.php/rmcg...,http://rmcg.geociencias.unam.mx/index.php/rmcg...
1,Revista Mexicana de Ciencias Geológicas,I. Físico Matemáticas y Ciencias de la Tierra,Geociencias,Early Cretaceous tectono-sedimentary evolution...,None,The Bisbee basin is considered to be the prima...,http://rmcg.geociencias.unam.mx/index.php/rmcg...,http://rmcg.geociencias.unam.mx/index.php/rmcg...
2,Revista Mexicana de Ciencias Geológicas,I. Físico Matemáticas y Ciencias de la Tierra,Geociencias,Structural evolution of the Sierra de Juárez m...,None,"The western side of Sierra de Juárez, State of...",http://rmcg.geociencias.unam.mx/index.php/rmcg...,http://rmcg.geociencias.unam.mx/index.php/rmcg...
3,Revista Mexicana de Ciencias Geológicas,I. Físico Matemáticas y Ciencias de la Tierra,Geociencias,Origin of the magnetite boundary in the Penins...,None,In the Peninsular Ranges of southern Californi...,http://rmcg.geociencias.unam.mx/index.php/rmcg...,http://rmcg.geociencias.unam.mx/index.php/rmcg...
4,Revista Mexicana de Ciencias Geológicas,I. Físico Matemáticas y Ciencias de la Tierra,Geociencias,"Las Américas Formation, Tlalpujahua, Michocán,...",None,The early Pliocene Las Américas Formation cons...,http://rmcg.geociencias.unam.mx/index.php/rmcg...,http://rmcg.geociencias.unam.mx/index.php/rmcg...
...,...,...,...,...,...,...,...,...
256,Revista Mexicana de Ciencias Geológicas,I. Físico Matemáticas y Ciencias de la Tierra,Geociencias,Un paleolago del NW mexicano en su conjunto pa...,None,El estudio de las oscilaciones climáticas reci...,http://rmcg.geociencias.unam.mx/index.php/rmcg...,http://rmcg.geociencias.unam.mx/index.php/rmcg...
257,Revista Mexicana de Ciencias Geológicas,I. Físico Matemáticas y Ciencias de la Tierra,Geociencias,Observaciones generales sobre la estructura de...,None,A partir de I`étude de coupes levées sur le po...,http://rmcg.geociencias.unam.mx/index.php/rmcg...,http://rmcg.geociencias.unam.mx/index.php/rmcg...
258,Revista Mexicana de Ciencias Geológicas,I. Físico Matemáticas y Ciencias de la Tierra,Geociencias,Mamíferos meocénicos de México: contribución a...,None,The Tertiary mammalian fauna of Mexico is smal...,http://rmcg.geociencias.unam.mx/index.php/rmcg...,http://rmcg.geociencias.unam.mx/index.php/rmcg...
259,Revista Mexicana de Ciencias Geológicas,I. Físico Matemáticas y Ciencias de la Tierra,Geociencias,Caracteres químicos principales del vulcanismo...,None,Les soixante analyses chimiques nouvelles prés...,http://rmcg.geociencias.unam.mx/index.php/rmcg...,http://rmcg.geociencias.unam.mx/index.php/rmcg...


In [132]:
df_catalogo.to_csv('Revista15.04.csv', index=False)